## What is NeoFS?

NeoFS is a distributed object storage service within the Neo blockchain ecosystem. It's a decentralized storage network that allows users to store and share data without relying on centralized service providers. NeoFS has the following features:

1. **Decentralized Storage**: Data is distributed across network nodes with no single point of failure.
2. **Data Ownership**: Users maintain complete control over their data.
3. **Blockchain Integration**: Tightly integrated with the Neo blockchain, providing security and auditability.
4. **Flexible Access Control**: Offers fine-grained access control policies.
5. **Data Integrity**: Uses cryptographic techniques to ensure data integrity and authenticity.

In this tutorial, we'll learn how to interact with NeoFS using Python, including creating containers, retrieving container lists, and performing signature operations.

## Helper Functions

First, let's define some helper functions for encryption, encoding, and utility operations.

In [ ]:
# Import necessary libraries
import os
import json
import binascii
import httpx
from base64 import b64decode, b64encode
from hashlib import sha256

# These libraries need to be installed: pip install pycryptodome ecdsa base58
import base58
from Crypto.PublicKey import ECC
from Crypto.Hash import RIPEMD160
import ecdsa

In [ ]:
# Functions about cryptographic operations
def wif_to_private_key(wif: str) -> bytes:
    """Convert from WIF format to raw private key"""
    # Step 1: Decode WIF key using Base58
    decoded = base58.b58decode(wif)
    
    # Step 2: Validate the length of decoded data
    if len(decoded) != 38:
        raise ValueError("Invalid WIF length")
    
    # First byte is version, last 4 bytes are checksum
    # version_byte = decoded[0]  # Typically 0x80 for Bitcoin mainnet
    private_key_bytes = decoded[1:-5]  # Extract private key portion
    checksum = decoded[-4:]  # Extract checksum portion
    
    # Step 3: Calculate expected checksum
    hash1 = sha256(decoded[:-4]).digest()
    hash2 = sha256(hash1).digest()
    expected_checksum = hash2[:4]
    
    # Check if provided checksum matches calculated checksum
    if checksum != expected_checksum:
        raise ValueError("Invalid WIF checksum")
    
    return private_key_bytes

def hex_to_private_key(hex_str: str) -> bytes:
    """Convert from hex string to private key bytes"""
    if hex_str.startswith('0x'):
        hex_str = hex_str[2:]
    return binascii.unhexlify(hex_str)

def private_key_to_neo3_public_key_and_address(private_key: bytes) -> (str, str):
    """Generate NEO3 public key and address from private key"""
    # Construct public key from private key using secp256r1 curve
    public_key = ECC.construct(curve='secp256r1', d=int.from_bytes(private_key, 'big')).pointQ
    x = public_key.x.to_bytes(32, 'big')
    # Choose prefix based on y-coordinate parity (compressed format)
    prefix = b'\x02' if public_key.y % 2 == 0 else b'\x03'
    compressed_public_key = prefix + x
    
    # Generate verification script
    verification_script = b'\x0c\x21' + compressed_public_key + b'\x41\x56\xe7\xb3\x27'
    
    # Calculate script hash
    ripemd160 = RIPEMD160.new()
    ripemd160.update(sha256(verification_script).digest())
    script_hash = ripemd160.digest()
    
    # Generate address (Neo address prefix is 0x35)
    address = base58.b58encode_check(b'\x35' + script_hash).decode('utf-8')
    
    return compressed_public_key.hex(), address

def sign_message(private_key: bytes, message: str | bytes) -> bytes:
    """Sign a message using private key"""
    assert len(private_key) == 32
    if type(message) is str:
        message: bytes = bytes.fromhex(message)
    pk = ecdsa.SigningKey.from_string(private_key, curve=ecdsa.NIST256p, hashfunc=sha256)
    signature: bytes = pk.sign_deterministic(message)
    return signature

def verify_message_signature(public_key: str | bytes, message: str | bytes, signature: bytes) -> bool:
    """Verify message signature"""
    if type(public_key) is str:
        public_key: bytes = bytes.fromhex(public_key)
    assert len(public_key) == 33
    assert public_key.startswith(b'\x02') or public_key.startswith(b'\x03')
    if type(message) is str:
        message: bytes = bytes.fromhex(message)
    public_key: ecdsa.VerifyingKey = ecdsa.VerifyingKey.from_string(public_key, curve=ecdsa.NIST256p, hashfunc=sha256)
    result = public_key.verify(signature, message)
    return result

In [ ]:
# Functions about data format conversions
def num2VarInt(num: int) -> str:
    """Convert a number to variable-length integer hex string representation"""
    if num < 0xfd:
        return num2hexstring(num)
    elif num <= 0xffff:
        # uint16
        return "fd" + num2hexstring(num, 2, True)
    elif num <= 0xffffffff:
        # uint32
        return "fe" + num2hexstring(num, 4, True)
    else:
        # uint64
        return "ff" + num2hexstring(num, 8, True)

def num2hexstring(num: int, size: int = 1, little_endian=False) -> str:
    """Convert a number to a hex string"""
    if not isinstance(num, int):
        raise TypeError(f"num2hexstring expected a number but got {type(num)}.")
    if num < 0:
        raise ValueError(f"num2hexstring expected a positive integer but got {num}.")
    if size % 1 != 0:
        raise ValueError(f"num2hexstring expected a positive integer but got {num}.")
    if num > 2 ** 53 - 1:
        raise ValueError(f"num2hexstring expected a safe integer but got {num}.")
    
    size *= 2
    hexstring = hex(num)[2:]
    hexstring = hexstring.zfill(size)
    
    if little_endian:
        hexstring = reverse_hex(hexstring)
    return hexstring

def reverse_hex(hexstring: str) -> str:
    """Reverse a hex string (reverse by byte pairs)"""
    return ''.join(reversed([hexstring[i:i + 2] for i in range(0, len(hexstring), 2)]))

## Key Configuration

In NeoFS, all operations require signing with a Neo wallet's private key to verify identity. Here, we'll start with a WIF (Wallet Import Format) private key and obtain the corresponding public key and address.

**Note**: Always protect your private key and never expose it in public or insecure environments.

In [ ]:
# Set up private key - Replace with your own private key in actual use
PRIVATE_KEY_WIF = ""  # Enter your WIF format private key here

# Convert from WIF format to raw private key
private_key = wif_to_private_key(PRIVATE_KEY_WIF)

# Get corresponding public key and NEO3 address
public_key, address = private_key_to_neo3_public_key_and_address(private_key)

# Set up the base URL for the NeoFS gateway
BASE_URL = 'https://rest.t5.fs.neo.org/v1/'  # This is a testnet URL, use the appropriate URL for production

# Create an HTTP client
httpx_client = httpx.Client(base_url=BASE_URL, headers={"Content-Type": "application/json"})

## Example 1: Creating and Retrieving Containers

In NeoFS, all data is stored in containers. Containers are similar to "buckets" or "folders" in traditional cloud storage, defining storage policies and access control rules for data.

Below we'll demonstrate how to create new containers and then query containers owned by the current user.

### 1.1 Creating an Authentication Token

Before creating a container, we need to obtain an authentication token. This is how permission control is implemented in NeoFS.

In [ ]:
# Set up authentication request headers
auth_header = {
    'X-Bearer-Owner-Id': address,       # Token owner's address
    'X-Bearer-Lifetime': "10000",       # Token validity period (seconds)
    'X-Bearer-For-All-Users': "false",  # Whether valid for all users
}

# Set up authentication content - Define token permissions
auth_content = [
    {"name":"my-bearer-token","object":[{"action":"ALLOW","filters":[],"operation":"GET","targets":[{"keys":[],"role":"OTHERS"}]}]},
    {"container":{"verb":"PUT"},"name":"my token"}
]

# Send authentication request
try:
    auth_resp = httpx_client.post('auth', 
                                  headers=auth_header, 
                                  content=json.dumps(auth_content)).json()
    
    # Decode and sign tokens
    for t in auth_resp:
        t["decoded_token"] = b64decode(t["token"])
    
    for t in auth_resp:
        t["signed_token"] = sign_message(private_key, t["decoded_token"]).hex()
    
    # Verify signatures
    for t in auth_resp:
        assert verify_message_signature(public_key, t["decoded_token"], bytes.fromhex(t["signed_token"]))
    
    print("Authentication token created successfully:")
    print(auth_resp)
except Exception as e:
    print(f"Failed to create authentication token: {e}")

### 1.2 Creating a New Container

Now that we have an authentication token, we can use it to create a new container. Container creation requires a signature, so we need to prepare the signature data first.

In [ ]:
# Use the second token (for container creation)
msg = auth_resp[1]['token']

# Generate random salt (for increased security)
random_salt = os.urandom(16).hex()  # Generate random salt

# Prepare parameters and serialize
parameter_hex_string = (random_salt + msg).encode().hex()
assert len(parameter_hex_string) % 2 == 0  # Ensure even length
length_hex = num2VarInt(len(parameter_hex_string) // 2)
concatenated_string = length_hex + parameter_hex_string
serialized_transaction = '010001f0' + concatenated_string + '0000'  # Specific format for transaction serialization

# Sign transaction data
signature = sign_message(private_key, serialized_transaction)
assert verify_message_signature(public_key, serialized_transaction, signature)
signature_hex_str = signature.hex()

# Set request headers
httpx_client.headers.update({"Authorization": f"Bearer {auth_resp[1]['token']}"})
bearer_header = {
    'X-Bearer-Owner-Id': address,                     # Owner address
    'X-Bearer-Signature': signature_hex_str + random_salt,  # Signature + random salt
    'X-Bearer-Signature-Key': public_key,              # Public key
}

# Create container
try:
    create_container_resp = httpx_client.put(
        'containers?walletConnect=true&name-scope-global=true',  # API endpoint and parameters
        headers=bearer_header,
        content=json.dumps(
            {
                "basicAcl": "public-read-write",     # Access control level
                "containerName": "MyFirstContainer",  # Container name
                "placementPolicy": "REP 3"           # Replication policy: maintain 3 copies
            }
        )
    )
    print(f"Container creation response: {create_container_resp.json()}")
    
    # Retrieve container list again to see newly created container
    list_container_resp = httpx_client.get(f'containers?ownerId={address}')
    print(f"\nUpdated container list: {list_container_resp.json()}")
    
except Exception as e:
    print(f"Failed to create container: {e}")

### 1.3 Retrieving the Current User's Container List

Now that we've created a container, let's see how to retrieve a list of all containers owned by the current user:

In [ ]:
# Get all containers owned by current user
try:
    my_containers = httpx_client.get(f'containers?ownerId={address}').json()
    print(f'My container list: {my_containers}')
except Exception as e:
    print(f"Failed to retrieve container list: {e}")

## Example 2: Signature Operations

In NeoFS, many operations require signatures to verify identity and authorization. The example below demonstrates how to sign a simple message ("hello"). This can be used as a test to verify that your key setup is correct, and to understand the signature mechanism in NeoFS.

In [ ]:
# Display public key and address
print(f"Public key: {public_key}")
print(f"Address: {address}")
print()

# Message to sign
msg = 'hello'

# Use a fixed random salt for verification and testing
# In production, use os.urandom(16).hex() to generate a random salt
random_salt = '2b62f24c77ec30bac716b116651b9d23'  

# Prepare parameters and serialize - similar to container creation process
parameter_hex_string = (random_salt + msg).encode().hex()
assert len(parameter_hex_string) % 2 == 0
length_hex = num2VarInt(len(parameter_hex_string) // 2)
concatenated_string = length_hex + parameter_hex_string
serialized_transaction = '010001f0' + concatenated_string + '0000'
print(f"Serialized transaction: {serialized_transaction}")
print()

# Sign transaction data
signature = sign_message(private_key, serialized_transaction)

# Output results
print(f"Random salt: {random_salt}")
print(f"Signature: {signature.hex()}")

## Additional NeoFS Operations

NeoFS provides many other functionalities, including but not limited to:

1. **Upload Objects**: Upload files to containers
2. **Download Objects**: Retrieve files from containers
3. **List Objects**: Get a list of files in a container
4. **Delete Objects**: Remove files from a container
5. **Set ACL**: Configure access control lists
6. **Create Extended ACL**: More complex access control policies

These operations follow a similar pattern: create an authentication token, sign the request, then perform the operation.
As you deepen your understanding of NeoFS, you can explore more advanced features such as setting up complex access control policies, managing large file uploads and downloads, implementing encrypted storage, and more.